<a href="https://colab.research.google.com/github/sanae-a11y/AI/blob/main/CreditScoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the librairies**


In [46]:
import matplotlib.pyplot as plt
from numpy import *
import pandas as pd

**Import the dataset**

In [41]:
file = pd.read_csv("CreditScoring.csv")

In [43]:
file

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,NaN
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN
3,1,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,0,88900,57264.0,90185.0,DebtCon,Other,16.0,0.0,0.0,221.808718,0.0,16.0,36.112347
5956,0,89000,54576.0,92937.0,DebtCon,Other,16.0,0.0,0.0,208.692070,0.0,15.0,35.859971
5957,0,89200,54045.0,92924.0,DebtCon,Other,15.0,0.0,0.0,212.279697,0.0,15.0,35.556590
5958,0,89800,50370.0,91861.0,DebtCon,Other,14.0,0.0,0.0,213.892709,0.0,16.0,34.340882


In [7]:
file.shape

(5960, 13)

In [14]:
data_types

BAD          int64
LOAN         int64
MORTDUE    float64
VALUE      float64
REASON      object
JOB         object
YOJ        float64
DEROG      float64
DELINQ     float64
CLAGE      float64
NINQ       float64
CLNO       float64
DEBTINC    float64
dtype: object

**Missing Values**

In [55]:
MissingData=file.isnull().sum().rename_axis('Variables').reset_index(name='Missing Values') # the isnull() returns 1 if the value is null
MissingData

,Variables,Missing Values
0,BAD,0
1,LOAN,0
2,MORTDUE,518
3,VALUE,112
4,REASON,252
5,JOB,279
6,YOJ,515
7,DEROG,708
8,DELINQ,580
9,CLAGE,308


In [15]:
def Entropie(S):
    S= array(S)
    classes_distinctes = set(S[:, 0])
    classes = list(S[:, 0])
    s = 0
    for c in classes_distinctes:
        p = classes.count(c)/len(classes)
        s += p*log2(p)
    return(-1.0*s)


In [18]:
print(Entropie(file))

0.7205815177959121


In [19]:
def GINI(S):
    S = array(S)
    clonne_des_classes = list(S[:, -1])
    classes = list(set(S[:, -1]))
    k = len(classes)
    s = 0
    for i in range(k):
        pi = clonne_des_classes.count(classes[i]/len(S[:, -1]))
        s += pi**2
    return 1-s

In [20]:
GINI(file)

1

In [53]:
BAD = file.iloc[:,0]
LOAN = file.iloc[:,1]
MORTDUE = file.iloc[:,1]
type(BAD)

pandas.core.series.Series

In [21]:
def Mesure_desorde(S, numero_colon_attribut):
    Bonne_mesure_desorde = + inf
    Bonne_valeur_de_repartition = 0
    S = S.to_numpy()
    classes = list(set(S[:, 0]))
    valeurs_attribut = list(set(S[:, numero_colon_attribut]))
    valeurs_attribut.sort()
    Gauche = []
    Droite = []

    for i in range(len(valeurs_attribut)-1):
        v = valeurs_attribut[i]
        Gauche.clear()
        Droite.clear()
        for e in S:
            val_attribut_pour_e = e[numero_colon_attribut]
            if val_attribut_pour_e <= v:
                Gauche.append(e)
            else:
                Droite.append(e)
        pGau = len(Gauche)/len(S)
        pDr = len(Droite)/len(S)
        mesure = (pGau)*GINI(Gauche)+(pDr)*GINI(Droite)
        if mesure < Bonne_mesure_desordre:
            Bonne_mesure_desordre = mesure
            Bonne_valeur_de_repartition = v
    return(Bonne_mesure_desordre, Bonne_valeur_de_repartition)

In [22]:
def Gain_d_information(S, numero_colon_attribut):
    S = S.to_numpy()
    classes = list(set(S[:, 0]))
    valeurs_attribut = list(set(S[:, numero_colon_attribut]))
    Si = [[]for i in range(len(valeurs_attribut))]
    for e in S:
        e = list(e)
        val_attribut_pour_e = e[numero_colon_attribut]
        numero_sous_ensemble = valeurs_attribut.index(val_attribut_pour_e)
        Si[numero_sous_ensemble].append(e)
    Si = array(Si)
    som = 0
    for sous_ensemble in Si:
        som += (len(sous_ensemble)/len(S))*Entropie(sous_ensemble)
    return(Entropie(S)-som)

In [23]:
print("LOAN :",Gain_d_information(file, 1))
print("MORTDUE :",Gain_d_information(file, 2))
print("REASON :",Gain_d_information(file, 3))
print("JOB :",Gain_d_information(file, 4))
print("YOJ :",Gain_d_information(file, 5))
print("DEROG :",Gain_d_information(file, 6))
print("DELINQ :",Gain_d_information(file, 7))
print("CLAGE :",Gain_d_information(file, 8))
print("NINQ :",Gain_d_information(file, 9))
print("CLNCO :",Gain_d_information(file, 10))
print("DEBTING :",Gain_d_information(file, 11))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


LOAN : 0.13247031605795323
MORTDUE : 0.6599670262166042
REASON : 0.6490620261099311
JOB : 0.0010200707812508814
YOJ : 0.014172286113719235
DEROG : 0.09890515375629483
DELINQ : 0.11308343379194186
CLAGE : 0.12913853183220558
NINQ : 0.6689316114488737
CLNCO : 0.07307049238021024
DEBTING : 0.05943267670920016
